# Problem 1 - Assignment 3 - GPUs
Approximate π using the Bailey-Borwein-Plouffe formula. $$\pi = \sum_{k=0}^{\infty} \left[ \frac{1}{16^k} \left( \frac{4}{8k+1} - \frac{2}{8k+4} - \frac{1}{8k+5} - \frac{1}{8k+6} \right) \right]$$

* Go to "Runtime" in the top menu.
* Select "Change runtime type".*
* Choose "T4 GPU" from the "Hardware accelerator" dropdown menu.
* Click "SAVE".

In [ ]:
#  verify GPU availability and its specifications
!nvidia-smi

Thu Mar 21 14:33:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#IMPORTS
import time
import matplotlib.pyplot as plt
import numpy as np

## Sequential

In [ ]:
def approximate_pi(n):
    pi = 0.0
    for i in range(n):
        pi += ((1/16) ** i) * (4 / (8 * i+ 1) - 2 / (8 * i + 4) - 1 / (8 * i + 5) - 1 / (8 * i + 6))
    return pi

n = 100000000

start_time_cpu = time.time()
pi_value = approximate_pi(n)
end_time_cpu = time.time()
elapsed_time_cpu = end_time_cpu - start_time_cpu

print("Number of iterations: ", n)
print("Approximation of π using BBP formula with CPU:", pi_value)
print("CPU Runtime:", elapsed_time_cpu, "s.")

Number of iterations:  100000000
Approximation of π using BBP formula with CPU: 3.141592653589793
CPU Runtime: 69.4721291065216 s.


In [ ]:
def get_time_cpu(n):
  start_time = time.time()
  pi_value = approximate_pi(n)
  end_time = time.time()
  return end_time - start_time

def avg_time_cpu(n_avg, n_range, outlier_cutoff=0.1):
    all_time_results = []

    # Collect all computation times
    for n in n_range:
        times_for_n = []
        for i in range(n_avg):
            times_for_n.append(get_time_cpu(n))
        all_time_results.append(times_for_n)
    # Compute the average
    avg_times = [sum(times) / len(times) for times in all_time_results]

    return avg_times

show_graph = False
n_avg = 10;
n_range = np.arange(100, 100000, 1000)

if (show_graph):
  # Generate results for different n
  pi_results_cpu = avg_time_cpu(n_avg, n_range)

  # Plot the results
  plt.figure(figsize=(5, 3))
  plt.plot(n_range, pi_results_cpu, marker='.', linestyle='-')
  plt.title("Approximation of π using BBP formula with CPU")
  plt.xlabel("Number of Terms (n)")
  plt.ylabel("Time")
  plt.grid(True)
  plt.show()

## Numpy

In [ ]:
import numpy as np

def approximate_pi_numpy(n):
    i = np.arange(n)
    return np.sum(((1/16) ** i) * (4 / (8 * i+ 1) - 2 / (8 * i + 4) - 1 / (8 * i + 5) - 1 / (8 * i + 6)))


start_time_numpy = time.time()
pi_value_numpy = approximate_pi_numpy(n)
end_time_numpy = time.time()
elapsed_time_numpy = end_time_numpy - start_time_numpy

print("Number of iterations: ", n)
print("Approximation of π using BBP formula with NumPy:", pi_value_numpy)
print("NumPy Runtime:", elapsed_time_numpy, "s.")
print(elapsed_time_cpu/elapsed_time_numpy, "times quicker.")

Number of iterations:  100000000
Approximation of π using BBP formula with NumPy: 3.141592653589793
NumPy Runtime: 5.770061492919922 s.
12.040102032147573 times quicker.


In [ ]:
def get_time_numpy(n):
  start_time = time.time()
  pi_value = approximate_pi_numpy(n)
  end_time = time.time()
  return end_time - start_time

def avg_time_numpy(n_avg, n_range):
    total_time_results = [0] * len(n_range)
    for j, n in enumerate(n_range):
      #print(n)
      for i in range(n_avg):
          #print(i, n)
          total_time_results[j] += get_time_numpy(n)
    return [time / n_avg for time in total_time_results]

from tqdm import tqdm
import time

def avg_time_numpy_progress(n_avg, n_range):
    total_time_results = [0] * len(n_range)
    for j, n in enumerate(n_range):
        progress_bar = tqdm(range(n_avg), desc=f'n = {n}', position=0, leave=True)
        for _ in progress_bar:
            total_time_results[j] += get_time_numpy(n)
            progress_bar.update(1)
            time.sleep(0.1)  # Add a small delay for visualization purposes (optional)
        progress_bar.close()
    return [time / n_avg for time in total_time_results]


if (show_graph):
  # Generate results for different n
  n_avg = 20;
  n_range = np.arange(1000, 10000, 1000)

  pi_results_numpy = avg_time_numpy_progress(n_avg, n_range)

  # Plot the results
  plt.figure(figsize=(5, 3))
  plt.plot(n_range, pi_results_numpy, marker='.', linestyle='-')
  plt.title("Approximation of π using BBP formula with numpy")
  plt.xlabel("Number of Terms (n)")
  plt.ylabel("Time")
  plt.grid(True)
  plt.show()

## CUPY

In [ ]:
import cupy as cp

def approximate_pi_cupy(n):
    i = cp.arange(n)
    return cp.sum(((1/16) ** i) * (4 / (8 * i+ 1) - 2 / (8 * i + 4) - 1 / (8 * i + 5) - 1 / (8 * i + 6)))


start_time_cupy = time.time()
pi_value_cupy = approximate_pi_cupy(n)
end_time_cupy = time.time()
elapsed_time_cupy = end_time_cupy - start_time_cupy

print("Number of iterations: ", n)
print("Approximation of π using BBP formula with CuPy:", pi_value_cupy)
print("CuPy Runtime:", elapsed_time_cupy, "s.")
print(elapsed_time_numpy/elapsed_time_cupy, "times quicker than numpy.")

Number of iterations:  100000000
Approximation of π using BBP formula with CuPy: 3.1415926535897936
CuPy Runtime: 0.004259586334228516 s.
1354.6060673905743 times quicker than numpy.


## Graph

In [ ]:
def get_time_cupy(n):
  start_time = time.time()
  pi_value = approximate_pi_cupy(n)
  end_time = time.time()
  return end_time - start_time

def avg_time_cupy(n_avg, n_range):
    total_time_results = [0] * len(n_range)
    for j, n in enumerate(n_range):
      for i in range(n_avg):
          #print(i, n)
          total_time_results[j] += get_time_cupy(n)
    return [time / n_avg for time in total_time_results]

def avg_time_cupy_progress(n_avg, n_range):
    total_time_results = [0] * len(n_range)
    for j, n in enumerate(n_range):
        progress_bar = tqdm(range(n_avg), desc=f'n = {n}', position=0, leave=True)
        for _ in progress_bar:
            total_time_results[j] += get_time_cupy(n)
            progress_bar.update(1)
            time.sleep(0.1)  # Add a small delay for visualization purposes (optional)
        progress_bar.close()
    return [time / n_avg for time in total_time_results]

In [ ]:
# Generate results for different n
n_avg = 10;
n_range = np.arange(10000, 100000000, 10000000)

if (show_graph):
  pi_results_numpy = avg_time_numpy_progress(n_avg, n_range)
  pi_results_cupy = avg_time_cupy_progress(n_avg, n_range)

  # Plot the results
  plt.figure(figsize=(5, 3))
  plt.plot(n_range, pi_results_numpy, marker='.', linestyle='-', label = 'numpy')
  plt.plot(n_range, pi_results_cupy, marker='.', linestyle='-', label = 'cupy')
  plt.title("Approximation of π using BBP formula with numpy and cupy")
  plt.xlabel("Number of Terms (n)")
  plt.ylabel("Time")
  plt.grid(True)
  plt.legend()
  plt.show()

##JIT (nopython)

In [ ]:
from numba import jit

@jit(nopython=True)
def j_approximate_pi(n):
    pi = 0.0
    for i in range(n):
        pi += ((1/16) ** i) * (4 / (8 * i+ 1) - 2 / (8 * i + 4) - 1 / (8 * i + 5) - 1 / (8 * i + 6))
    return pi


start_time_j = time.time()
pi_value_j = j_approximate_pi(n)
end_time_j = time.time()
elapsed_time_j = end_time_j - start_time_j

print("Number of iterations: ", n)
print("Approximation of π using BBP formula with JIT:", pi_value_j)
print("JIT Runtime:", elapsed_time_j, "s.")

Number of iterations:  100000000
Approximation of π using BBP formula with JIT: 3.141592653589793
JIT Runtime: 1.675957441329956 s.


## JIT (nopython + parallel)

In [ ]:
from numba import prange

@jit(nopython=True, parallel=True)
def jp_approximate_pi(n):
    pi = 0.0
    for i in prange(n):
        pi += ((1/16) ** i) * (4 / (8 * i+ 1) - 2 / (8 * i + 4) - 1 / (8 * i + 5) - 1 / (8 * i + 6))
    return pi


start_time_jp = time.time()
pi_value_jp = jp_approximate_pi(n)
end_time_jp = time.time()
elapsed_time_jp = end_time_jp - start_time_jp

print("Number of iterations: ", n)
print("Approximation of π using BBP formula with JIT + parallel:", pi_value_jp)
print("JIT + parallel Runtime:", elapsed_time_jp, "s.")

Number of iterations:  100000000
Approximation of π using BBP formula with JIT + parallel: 3.141592653589793
JIT + parallel Runtime: 1.636348009109497 s.


# TimeIt

In [ ]:
n_range = [100000, 1000000, 10000000, 100000000]
for n in n_range:
  print("\n \033[1mNumber of iterations: \033[0m", n)
  print("\nSequential version:")
  %timeit approximate_pi(n)

  print("\nNumpy version:")
  %timeit approximate_pi_numpy(n)

  print("\nCupy version:")
  %timeit approximate_pi_cupy(n)

  print("\nno-python = true")
  %timeit j_approximate_pi(n)

  print("\nno-python = true, parallel = true")
  %timeit jp_approximate_pi(n)


 Number of iterations:  100000

Sequential version:
56.7 ms ± 649 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Numpy version:
3.77 ms ± 76.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Cupy version:
567 µs ± 105 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

no-python = true
8.65 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

no-python = true, parallel = true
8.25 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

 Number of iterations:  1000000

Sequential version:
566 ms ± 6.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Numpy version:
61.8 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Cupy version:
2.31 ms ± 64.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

no-python = true
16.2 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

no-python = true, parallel = true
15.3 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

 Number of ite